In [1]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm, dlm], topics, qrels, 
                                names=['tuned_dlm','orignal_dlm'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20original_tuned_dlm', '20orignal_dlm'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21original_tuned_dlm', '21orignal_dlm'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            tuned_dlm.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/original_tuned_dlm.csv")
#             tuned_dlm.to_csv(path_or_buf=f"/workspace/src/runs_dlm/original_tuned_dlm.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.652732
Best setting is ['BR(DirichletLM) dirichletlm.mu=3000']
Fold 2
Best ndcg_cut_5 is 0.513757
Best setting is ['BR(DirichletLM) dirichletlm.mu=750']


/tmp/ipykernel_3689098/4171921949.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics, qrels,


\begin{tabular}{llrrr}
\toprule
{} &         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    tuned\_dlm &    0.569517 &     0.466434 &  0.594445 \\
1 &  orignal\_dlm &    0.573641 &     0.452373 &  0.590270 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/4171921949.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20original_tuned_dlm', '20orignal_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20original\_tuned\_dlm &    0.502843 &     0.381891 &  0.573064 \\
1 &         20orignal\_dlm &    0.506382 &     0.386060 &  0.581117 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/4171921949.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21original_tuned_dlm', '21orignal_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21original\_tuned\_dlm &    0.634858 &     0.549287 &  0.615399 \\
1 &         21orignal\_dlm &    0.639554 &     0.517360 &  0.599241 \\
\bottomrule
\end{tabular}



In [3]:
'''
Example of simple tuning dlm and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm, dlm], topics, qrels, names=[folders[i] + '_tuned_dlm', folders[i] + '_orignal_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm', '20' + folders[i] + '_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm', '21'+ folders[i] + '_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            tuned_dlm.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm.csv")
#             tuned_dlm.to_csv(path_or_buf=f"/workspace/src/runs_dlm/{folders[i]}_tuned_dlm.csv")
            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.607973
Best setting is ['BR(DirichletLM) dirichletlm.mu=3750']
Fold 2
Best ndcg_cut_5 is 0.450963
Best setting is ['BR(DirichletLM) dirichletlm.mu=5750']


/tmp/ipykernel_3689098/2513816004.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics, qrels, names=[folders[i] + '_tuned_dlm', folders[i] + '_orignal_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    conclusions\_tuned\_dlm &    0.525312 &     0.444128 &  0.700047 \\
1 &  conclusions\_orignal\_dlm &    0.524059 &     0.448322 &  0.705230 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/2513816004.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm', '20' + folders[i] + '_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                    name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20conclusionstuned\_dlm &    0.448751 &     0.373309 &  0.680320 \\
1 &       20conclusions\_dlm &    0.441318 &     0.370764 &  0.683595 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    pools\_conclusions\_tuned\_dlm &    0.516184 &     0.435566 &  0.696440 \\
1 &  pools\_conclusions\_orignal\_dlm &    0.517417 &     0.435083 &  0.697605 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/2513816004.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm', '20' + folders[i] + '_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20pools\_conclusionstuned\_dlm &    0.414085 &     0.349236 &  0.677383 \\
1 &       20pools\_conclusions\_dlm &    0.416576 &     0.348261 &  0.679737 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/2513816004.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm', '21'+ folders[i] + '_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21pools\_conclusionstuned\_dlm &     0.61624 &     0.520169 &  0.715116 \\
1 &       21pools\_conclusions\_dlm &     0.61624 &     0.520169 &  0.715116 \\
\bottomrule
\end{tabular}



In [4]:
'''
Example of simple tuning dlm and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm, dlm], topics, qrels, names=[folders[i] + '_tuned_dlm', folders[i] + '_orignal_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm', '20' + folders[i] + '_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm', '21'+ folders[i] + '_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            tuned_dlm.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm.csv")
#             tuned_dlm.to_csv(path_or_buf=f"/workspace/src/runs_dlm/{folders[i]}_tuned_dlm.csv")
            i += 1
            


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.455074
Best setting is ['BR(DirichletLM) dirichletlm.mu=1000']
Fold 2
Best ndcg_cut_5 is 0.250555
Best setting is ['BR(DirichletLM) dirichletlm.mu=1500']


/tmp/ipykernel_3689098/676691230.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics, qrels, names=[folders[i] + '_tuned_dlm', folders[i] + '_orignal_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    marco-pretrained\_tuned\_dlm &    0.348440 &     0.311509 &  0.610536 \\
1 &  marco-pretrained\_orignal\_dlm &    0.339255 &     0.310085 &  0.608835 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/676691230.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm', '20' + folders[i] + '_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20marco-pretrainedtuned\_dlm &    0.247279 &     0.252619 &  0.601926 \\
1 &       20marco-pretrained\_dlm &    0.235450 &     0.248456 &  0.599014 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689098/676691230.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm', '21'+ folders[i] + '_dlm'],


\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21marco-pretrainedtuned\_dlm &    0.447577 &      0.36922 &  0.618973 \\
1 &       21marco-pretrained\_dlm &    0.440984 &      0.37048 &  0.618459 \\
\bottomrule
\end{tabular}

